In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Use requests to parse wikipedia page
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#Get the page as text
data = r.text

#Import data to beautiful soup
soup = BeautifulSoup(data)

#find the table
table = soup.find_all('table')[0]

#parse the data of the table
df = pd.read_html(str(table), header=0)[0]

#Clean Borough values that are not assigned
df = df[df['Borough'] != "Not assigned"]

#Translate Neighbourhood values to Borough when they are not assigned
df.loc[df['Neighbourhood'] == "Not assigned",'Neighbourhood'] = df['Borough']

#Group Neighbourhood values that belong to the same borough
df2 =  pd.DataFrame(df.groupby(['Postcode'],as_index=True)['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)))
df.set_index('Postcode', inplace=True,)

#Drop Neighbourhood column because we will get it from the other frame
df.drop(columns=['Neighbourhood'], inplace=True)

#Join the two dataframe into one by postcode as index
df3 = df2.join(df, how='outer')

#Drop duplicates
df3.drop_duplicates(inplace=True)
df = df3[['Borough','Neighbourhood']]

#Print results
df.shape


(103, 2)

In [2]:
import pandas as pd

#Parse coordinates from csv
dftest = pd.read_csv('Geospatial_Coordinates.csv')

#make postal code column index in order to join the other table
dftest.set_index('Postal Code', inplace=True,)
dftest = df.join(dftest, how='outer')
dftest.reset_index(level=0, inplace=True)
dftest

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [3]:
import folium

#Build the map of Toronto
m = folium.Map([43.653963, -79.387207], zoom_start=11)



# mark each pair of coordinates as a point
for index, row in dftest.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=6,
                        fill_color="#3db7e4",
                       ).add_to(m)
#view map
m
